In [23]:
# Python library imports: numpy, random, sklearn, pandas, etc

import warnings
warnings.filterwarnings('ignore')

import sys
import random
import numpy as np

# cross_validation is deprecated since version 0.18. This module will be removed in 0.20.
# Use sklearn.model_selection.train_test_split instead.
# from sklearn import linear_model, cross_validation, metrics, svm
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics, svm
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
%matplotlib inline

In [21]:
# Config size
TRAINING_MATRIX_RESULT_FILE_NAME = 'training_matrix_result.pickle'

In [22]:
# from date import date
from datetime import date
# # Python UDFs for our PIG script # from date import date # get hour-of-day from HHMM field @outputSchema("value: int") def get_hour(val): return int(val.zfill(4)[:2]) # this array defines the dates of holiday in 2007 and 2008
# holidays = [date(2007, 1, 1), date(2007, 1, 15), date(2007, 2, 19), date(2007, 5, 28), date(2007, 6, 7), date(2007, 7, 4), \ date(2007, 9, 3), date(2007, 10, 8), date(2007, 11, 11), date(2007, 11, 22), date(2007, 12, 25), 
#             date(2008, 1, 1), date(2008, 1, 21), date(2008, 2, 18), date(2008, 5, 22), date(2008, 5, 26), date(2008, 7, 4), \ date(2008, 9, 1), date(2008, 10, 13), date(2008, 11, 11), date(2008, 11, 27), date(2008, 12, 25) \ ] # get number of days from nearest holiday @outputSchema("days: int") def days_from_nearest_holiday(year, month, day): d = date(year, month, day) x = [(abs(d-h)).days for h in holidays] return min(x)

holiday_list = []

# Hard code holiday
holiday_1987 = [date(1987, 1, 1), date(1987, 1, 19), date(1987, 2, 16), date(1987, 5, 1), date(1987, 5, 25), date(1987, 7, 4), date(1987, 9, 7), date(1987, 10, 12), date(1987, 11, 11), date(1987, 11, 26), date(1987, 12, 25)]
holiday_1988 = [date(1988, 1, 1), date(1988, 1, 18), date(1988, 2, 15), date(1988, 5, 1), date(1988, 5, 30), date(1988, 7, 4), date(1988, 9, 5), date(1988, 10, 10), date(1988, 11, 11), date(1988, 11, 24), date(1988, 12, 25)]
holiday_1989 = [date(1989, 1, 1), date(1989, 1, 16), date(1989, 2, 20), date(1989, 5, 1), date(1989, 5, 29), date(1989, 7, 4), date(1989, 9, 4), date(1989, 10, 9), date(1989, 11, 11), date(1989, 11, 23), date(1989, 12, 25)]
holiday_1990 = [date(1990, 1, 1), date(1990, 1, 15), date(1990, 2, 19), date(1990, 5, 1), date(1990, 5, 28), date(1990, 7, 4), date(1990, 9, 3), date(1990, 10, 8), date(1990, 11, 11), date(1990, 11, 22), date(1990, 12, 25)]
holiday_1991 = [date(1991, 1, 1), date(1991, 1, 21), date(1991, 2, 18), date(1991, 5, 1), date(1991, 5, 27), date(1991, 7, 4), date(1991, 9, 2), date(1991, 10, 14), date(1991, 11, 11), date(1991, 11, 28), date(1991, 12, 25)]
holiday_1992 = [date(1992, 1, 1), date(1992, 1, 20), date(1992, 2, 17), date(1992, 5, 1), date(1992, 5, 25), date(1992, 7, 4), date(1992, 9, 7), date(1992, 10, 12), date(1992, 11, 11), date(1992, 11, 26), date(1992, 12, 25)]
holiday_1993 = [date(1993, 1, 1), date(1993, 1, 18), date(1993, 2, 15), date(1993, 5, 1), date(1993, 5, 31), date(1993, 7, 4), date(1993, 9, 6), date(1993, 10, 11), date(1993, 11, 11), date(1993, 11, 25), date(1993, 12, 25)]
holiday_1994 = [date(1994, 1, 1), date(1994, 1, 17), date(1994, 2, 21), date(1994, 5, 1), date(1994, 5, 30), date(1994, 7, 4), date(1994, 9, 5), date(1994, 10, 10), date(1994, 11, 11), date(1994, 11, 24), date(1994, 12, 25)]
holiday_1995 = [date(1995, 1, 1), date(1995, 1, 16), date(1995, 2, 20), date(1995, 5, 1), date(1995, 5, 29), date(1995, 7, 4), date(1995, 9, 4), date(1995, 10, 9), date(1995, 11, 11), date(1995, 11, 23), date(1995, 12, 25)]
holiday_1996 = [date(1996, 1, 1), date(1996, 1, 15), date(1996, 2, 19), date(1996, 5, 1), date(1996, 5, 27), date(1996, 7, 4), date(1996, 9, 2), date(1996, 10, 14), date(1996, 11, 11), date(1996, 11, 28), date(1996, 12, 25)]

holiday_list.extend(holiday_1987)
holiday_list.extend(holiday_1988)
holiday_list.extend(holiday_1989)
holiday_list.extend(holiday_1990)
holiday_list.extend(holiday_1991)
holiday_list.extend(holiday_1992)
holiday_list.extend(holiday_1993)
holiday_list.extend(holiday_1994)
holiday_list.extend(holiday_1995)
holiday_list.extend(holiday_1996)

def days_from_nearest_holiday(year, month, day): 
    d = date(year, month, day) 
    x = [(abs(d-h)).days for h in holiday_list]
    return min(x)

### Load training matrix result

In [24]:
load_training_model_st = time.time()

# Load trained model from file
result_location_path = f"./training_results/{TRAINING_MATRIX_RESULT_FILE_NAME}"
clf_rf = pickle.load(open(result_location_path, "rb"))

# Load training model time
load_training_model_et = time.time()
elapsed_time = load_training_model_et - load_training_model_st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 22.72929096221924 seconds


### Import data and make performance test

In [25]:
load_data_st = time.time()
# read files
cols = ['dep_delay', 'month', 'day_of_month', 'day_of_week', 'dest',  'distance']
col_types = {'dep_delay': int, 'month': int, 'day_of_month': int, 'day_of_week': int, 'distance': int, 
            'dest': str}

df_1987 = pd.read_csv('result/1987.csv')
df_1988 = pd.read_csv('result/1988.csv')
df_1989 = pd.read_csv('result/1989.csv')
df_1990 = pd.read_csv('result/1990.csv')
df_1991 = pd.read_csv('result/1991.csv')
df_1992 = pd.read_csv('result/1992.csv')
df_1993 = pd.read_csv('result/1993.csv')
df_1994 = pd.read_csv('result/1994.csv')
df_1995 = pd.read_csv('result/1995.csv')
df_1996 = pd.read_csv('result/1996.csv')

# df_1996 = pd.read_csv('result/1996.csv')
# df_1996 = df_1996[cols]

df_years = [df_1987, df_1988, df_1989, df_1990, df_1991, df_1992, df_1993, df_1994, df_1995, df_1996]

flight_df = pd.concat(df_years)
flight_df

# Load data time
load_data_et = time.time()
elapsed_time = load_data_et - load_data_st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 82.4133472442627 seconds


In [26]:
process_hdays_st = time.time()

# Process holidays
flight_df['hdays'] = flight_df.apply(lambda row: days_from_nearest_holiday(row['year'], row['month'], row['day_of_month']), axis=1)

# elapsed time
process_hdays_et = time.time()
elapsed_time = process_hdays_et - process_hdays_st
print('Execution time:', elapsed_time, 'seconds')


Execution time: 1743.800420999527 seconds


In [33]:
# Select columns to reduce the training matrix side and define dimension for prediction feature
cols = ['unique_carrier', 'month', 'flight_num', 'day_of_month', 'day_of_week', 'dest', 'origin', 'dep_time', 'distance', 'dep_delay', 'hdays']
flight_df_selc = flight_df[cols]
# flight_df_selc['unique_carrier'].unique()
flight_df_selc["unique_carrier"].values.reshape(-1, 1)


array([['PS'],
       ['PS'],
       ['PS'],
       ...,
       ['AA'],
       ['AA'],
       ['AA']], dtype=object)

In [34]:
flight_df_selc['dep_delay'] = (flight_df_selc['dep_delay'] > 10) *1
categ_cols = ['unique_carrier', 'flight_num', 'dest', 'origin', 'hdays']

for item  in categ_cols:
    flight_df_selc[item] = flight_df_selc[item].astype('category').cat.codes + 1


In [35]:
# Divide test and train matrix
train, test, y_train, y_test = train_test_split(flight_df_selc.drop(['dep_delay'], axis=1), flight_df_selc["dep_delay"],
                                                random_state=10, test_size=0.2)

In [36]:
test["unique_carrier"].unique()

array([13,  4, 14, 15,  3,  1, 10, 12,  8,  6,  5,  2,  9, 11,  7],
      dtype=int8)

In [37]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(train["unique_carrier"].values.reshape(-1, 1))  

# enc.feature_indices_
airline_onehot = enc.transform(train["unique_carrier"].values.reshape(-1, 1)).toarray()
airline_onehot_test = enc.transform(test["unique_carrier"].values.reshape(-1, 1)).toarray()

In [38]:
air_oh_df = pd.DataFrame(airline_onehot, columns = ["A1","A2","A3","A4","A5","A6","A7",
                                                    "A8","A9","A10", "A11", "A12", "A13", "A14", "A15"])
air_oh_test_df = pd.DataFrame(airline_onehot_test, columns = ["A1","A2","A3","A4","A5","A6","A7",
                                                         "A8","A9","A10", "A11", "A12", "A13", "A14", "A15"])

In [39]:
train2 = pd.concat([train.reset_index(),air_oh_df.reset_index()],axis=1).drop(["index","month","day_of_month","day_of_week","unique_carrier","flight_num","dest","origin", "hdays"],axis=1)
test2  = pd.concat([test.reset_index(),air_oh_test_df.reset_index()],axis=1).drop(["index","month","day_of_month","day_of_week","unique_carrier","flight_num","dest","origin", "hdays"],axis=1)

test2

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
0,1617,494.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1029,719.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2053,595.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1248,331.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1145,487.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9393187,700,347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9393188,1540,2475.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9393189,700,1518.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9393190,1248,1671.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [40]:
# showing the performance of a classification model for binary
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))
# Parameter Tuning
param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
             "n_estimators": [200],
              "learning_rate": [0.05,0.1,0.16]}

### Evaluate on test set

In [41]:
predict_st = time.time()

# Evaluate on test set
pr = clf_rf.predict(test2)

# print results
cm = confusion_matrix(y_test, pr)
print("Confusion matrix")
print(pd.DataFrame(cm))
report_svm = precision_recall_fscore_support(list(y_test), list(pr), average='micro')
print("\nprecision = %0.2f, recall = %0.2f, F1 = %0.2f, accuracy = %0.2f\n" % \
        (report_svm[0], report_svm[1], report_svm[2], accuracy_score(list(y_test), list(pr))))


# save time
predict_et = time.time()
elapsed_time = predict_et - predict_st
print('Execution time:', elapsed_time, 'seconds')

Confusion matrix
         0       1
0  7508993  229761
1   794633  859805

precision = 0.89, recall = 0.89, F1 = 0.89, accuracy = 0.89

Execution time: 134.78062176704407 seconds


In [18]:
y_test2 = pd.DataFrame({"y":y_test.reset_index(drop=True)})
predict2 = pd.DataFrame({"yhat":pr})
x_test2 = pd.DataFrame(test2)
result = x_test2.join(y_test2)
predicted_result = result.join(predict2)
predicted_result

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,y,yhat
0,740,718.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
1,1635,1754.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,2002,305.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,725,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0
4,1401,956.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7125325,1734,733.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
7125326,1837,650.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0
7125327,1415,1754.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
7125328,1753,2106.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0


In [19]:
predicted_result[predicted_result['y'] != predicted_result['yhat']]

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,y,yhat
13,1420,1011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0
22,2135,239.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0
77,1818,440.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0
79,1751,1024.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0
93,1500,2677.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7125295,1217,1589.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
7125308,1726,1619.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
7125314,1551,1208.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
7125317,1717,732.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
